In [1]:
import random
import numpy as np
np.random.seed(0)
import pandas as pd
from sklearn.metrics import make_scorer, accuracy_score
import pickle

# Load the dataset
df = pd.read_csv("SongPopularity_classification_test.csv")

# Display the first few rows of the dataset
df.head()

,Song,Album,Album Release Date,Artist Names,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank,Song Length(ms),Spotify Link,Song Image,...,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Key,Mode,Time Signature
0,You Make Me Feel... (feat. Sabi),Night Shades,8/29/2011,"['Cobra Starship', 'Sabi']","['neon pop punk', 'post-teen pop', 'pop punk']",2011,48,215693,https://open.spotify.com/track/16Of7eeW44kt0a1...,https://i.scdn.co/image/ab67616d00001e02a4c018...,...,0.857,0.000007,0.0385,-2.944,0.0535,131.959,0.748,7,0,4
1,Beautiful Liar (Karaoke Version) - Originally ...,Remember Me - 2007,1/1/2012,['Sing Karaoke Sing'],['karaoke'],2007,62,198207,https://open.spotify.com/track/4l56dfITI1E1n0R...,https://i.scdn.co/image/ab67616d00001e029bb51b...,...,0.439,0.074800,0.2170,-9.475,0.0459,92.017,0.367,8,1,4
2,Electric Avenue (Karaoke in the style of Eddy ...,1983 Karaoke Classics Volume 1,9/21/2009,"[""1980's Karaoke Band""]",['karaoke'],1983,22,231346,https://open.spotify.com/track/3LiDpAoFjGJCtH8...,https://i.scdn.co/image/ab67616d00001e027e27cb...,...,0.303,0.000024,0.0562,-15.505,0.3630,119.843,0.787,11,0,4
3,Tarzan Boy,Living In The Background,1/1/1985,['Baltimora'],"['synthpop', 'italian disco']",1986,73,378946,https://open.spotify.com/track/273uCXd7NPrInai...,https://i.scdn.co/image/ab67616d00001e02de8220...,...,0.573,0.028000,0.3060,-12.300,0.0261,107.363,0.716,5,1,4
4,I Need to Know,Marc Anthony,9/28/1999,['Marc Anthony'],"['salsa', 'modern salsa', 'latin pop', 'tropic...",2000,23,227706,https://open.spotify.com/track/7ffwRz8lZyDOE4V...,https://i.scdn.co/image/ab67616d00001e02427f86...,...,0.949,0.000010,0.1120,-2.563,0.0336,115.061,0.792,3,0,4


In [2]:
# Remove duplicated
df = df[~df.duplicated()==1]


#Remove the Square Brackets from the artists

df["Artist Names"]=df["Artist Names"].str.replace("[", "")
df["Artist Names"]=df["Artist Names"].str.replace("]", "")
df["Artist Names"]=df["Artist Names"].str.replace("'", "")


df["Artist(s) Genres"]=df["Artist(s) Genres"].str.replace("[", "")
df["Artist(s) Genres"]=df["Artist(s) Genres"].str.replace("]", "")
df["Artist(s) Genres"]=df["Artist(s) Genres"].str.replace("'", "")

# Transform milliseconds to minutes
df["Song Length(mn)"] = df["Song Length(ms)"]/60000
df.drop(columns="Song Length(ms)", inplace=True)


In [3]:
# It is the selected features based on the correlation while the training phase
selected_features=['PopularityLevel', 'Hot100 Ranking Year', 'Loudness', 'Energy',
       'Artist(s) Genres', 'Speechiness', 'Danceability', 'Album Release Date',
       'Valence', 'Hot100 Rank', 'Acousticness']

df = df[selected_features]

In [4]:
df['PopularityLevel']=pd.Categorical(df['PopularityLevel']).codes

In [5]:
X= df.drop(['PopularityLevel'], axis=1)
y= df['PopularityLevel']

In [6]:
obj_columns = ['Album Release Date']

# Load the encoder model
with open('encode.pkl', 'rb') as f:
    e = pickle.load(f)
for col in obj_columns:
    # Encode the data and store it in a variable
    X_encoded = e.fit_transform(X[col])   
    
    X[col] = X_encoded  

# Now X_train contains the encoded data
X.head()

,Hot100 Ranking Year,Loudness,Energy,Artist(s) Genres,Speechiness,Danceability,Album Release Date,Valence,Hot100 Rank,Acousticness
0,2011,-2.944,0.857,"neon pop punk, post-teen pop, pop punk",0.0535,0.668,437,0.748,48,0.01910
1,2007,-9.475,0.439,karaoke,0.0459,0.764,47,0.367,62,0.00112
2,1983,-15.505,0.303,karaoke,0.3630,0.985,469,0.787,22,0.01620
3,1986,-12.300,0.573,"synthpop, italian disco",0.0261,0.684,23,0.716,73,0.31800
4,2000,-2.563,0.949,"salsa, modern salsa, latin pop, tropical",0.0336,0.813,482,0.792,23,0.48600


In [7]:
# Load the preprocessing model
with open('preprocessing.pkl', 'rb') as f:
    p = pickle.load(f)

p.fit(X)

# Transform the selected columns
X_preprocessed = p.transform(X)

# Retrieve the column names after transformation
transformed_feature_names = p.get_feature_names_out(input_features=X.columns)

# Convert the transformed array to a DataFrame
X_transformed = pd.DataFrame(X_preprocessed, columns=transformed_feature_names)

m=['Hot100 Ranking Year', "Artist(s) Genres",'Hot100 Rank']
# Drop the original categorical columns from X_train
X.drop(m, axis=1, inplace=True)

# Concatenate the encoded features with the original data
X = pd.concat([X, X_transformed], axis=1)

X   

,Loudness,Energy,Speechiness,Danceability,Album Release Date,Valence,Acousticness,minmax__Hot100 Ranking Year,minmax__Hot100 Rank,categorical__Artist(s) Genres_,...,"categorical__Artist(s) Genres_viral pop, pop, dance pop, canadian pop","categorical__Artist(s) Genres_viral pop, post-teen pop",categorical__Artist(s) Genres_vocal harmony group,"categorical__Artist(s) Genres_vocal harmony group, deep adult standards","categorical__Artist(s) Genres_vocal harmony group, easy listening","categorical__Artist(s) Genres_vocal harmony group, swing, adult standards, torch song",categorical__Artist(s) Genres_western swing,"categorical__Artist(s) Genres_western swing, bluegrass gospel, country boogie, yodeling, country gospel",categorical__Artist(s) Genres_yacht rock,"categorical__Artist(s) Genres_yacht rock, soft rock, mellow gold, singer-songwriter, classic rock"
0,-2.944,0.857,0.0535,0.668,437,0.748,0.01910,0.855263,0.474747,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-9.475,0.439,0.0459,0.764,47,0.367,0.00112,0.802632,0.616162,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-15.505,0.303,0.3630,0.985,469,0.787,0.01620,0.486842,0.212121,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-12.300,0.573,0.0261,0.684,23,0.716,0.31800,0.526316,0.727273,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.563,0.949,0.0336,0.813,482,0.792,0.48600,0.710526,0.222222,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,-2.353,0.786,0.0380,0.162,275,0.138,0.03740,0.802632,0.636364,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684,-8.918,0.520,0.0296,0.526,176,0.675,0.20200,0.302632,0.585859,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
685,-3.363,0.862,0.2550,0.375,487,0.478,0.07100,0.815789,0.363636,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
686,-5.181,0.860,0.4300,0.697,42,0.433,0.03680,0.802632,0.727273,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# It is the selected features based on the ANOVA while the training phase
selected_features_test=['Loudness', 'Energy', 'Speechiness', 'Danceability',
       'Album Release Date', 'Valence', 'Acousticness',
       'minmax__Hot100 Ranking Year', 'minmax__Hot100 Rank',
       'categorical__Artist(s) Genres_',
       'categorical__Artist(s) Genres_adult standards, easy listening',
       'categorical__Artist(s) Genres_country road, contemporary country, modern country rock, country',
       'categorical__Artist(s) Genres_deep adult standards',
       'categorical__Artist(s) Genres_doo-wop, rhythm and blues',
       'categorical__Artist(s) Genres_easy listening',
       'categorical__Artist(s) Genres_karaoke',
       'categorical__Artist(s) Genres_pop',
       'categorical__Artist(s) Genres_pop, dance pop',
       'categorical__Artist(s) Genres_rock-and-roll, adult standards, easy listening, cowboy western',
       'categorical__Artist(s) Genres_swing, vaudeville, deep adult standards, british dance band']
       
X = X[selected_features_test]

KeyError: "['categorical__Artist(s) Genres_rock-and-roll, adult standards, easy listening, cowboy western', 'categorical__Artist(s) Genres_swing, vaudeville, deep adult standards, british dance band'] not in index"

In [ ]:
with open('GradientBoostingClassifier.pkl', 'rb') as f:
         GBC= pickle.load(f)
        

with open('randomforestclassifier.pkl', 'rb') as f:
         RFC= pickle.load(f)


with open('svc.pkl', 'rb') as f:
         SVC= pickle.load(f)


# Evaluate models on test data (including ensemble)
def evaluate_model(model,model_name, X, y):
    
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print(f"Model: {model_name}, Accuracy: {accuracy}")

evaluate_model(RFC,"RFC", X, y)
evaluate_model(GBC, "GBC",X, y)
evaluate_model(SVC,"SVC" ,X, y)


In [ ]:
with open('Stacking_Model.pkl', 'rb') as f:
         Stacking_Model= pickle.load(f)

evaluate_model(Stacking_Model,"Stacking model",X, y)

In [ ]:
with open('Voting_Model.pkl', 'rb') as f:
         Voting_Model= pickle.load(f)

evaluate_model(Voting_Model,"Voting model", X, y)